# The vault lock

Apperently the last puzzle, hidden in the ruins on the north of beach.

## The founded journal

> Fireflies were using this dusty old journal as a resting spot until you scared them off.  It reads:
> 
> Day 1: We have reached what seems to be the final in a series of puzzles guarding an ancient treasure.  I suspect most adventurers give up long before this point, but we're so close!  We must press on!
> 
> Day 1: P.S.: It's a good thing the island is tropical.  We should have food for weeks!
> 
> Day 2: The vault appears to be sealed by a mysterious force - the door won't budge an inch.  We don't have the resources to blow it open, and I wouldn't risk damaging the contents even if we did.  We'll have to figure out the lock mechanism.
> 
> Day 3: The door to the vault has a number carved into it.  Each room leading up to the vault has more numbers or symbols embedded in mosaics in the floors.  We even found a strange glass orb in the antechamber on a pedestal itself labeled with a number.  What could they mean?
> 
> Day 5: We finally built up the courage to touch the strange orb in the antechamber.  It flashes colors as we carry it from room to room, and sometimes the symbols in the rooms flash colors as well.  It simply evaporates if we try to leave with it, but another appears on the pedestal in the antechamber shortly thereafter.  It also seems to do this even when we return with it to the antechamber from the other rooms.
> 
> Day 8: When the orb is carried to the vault door, the numbers on the door flash black, and then the orb evaporates.  Did we do something wrong?  Doesn't the door like us?  We also found a small hourglass near the door, endlessly running.  Is it waiting for something?
> 
> Day 13: Some of my crew swear the orb actually gets heaver or lighter as they walk around with it.  Is that even possible?  They say that if they walk through certain rooms repeatedly, they feel it getting lighter and lighter, but it eventually just evaporates and a new one appears as usual.
> 
> Day 21: Now I can feel the orb changing weight as I walk around.  It depends on the area - the change is very subtle in some places, but certainly more noticeable in others, especially when I walk into a room with a larger number or out of a room marked '*'.  Perhaps we can actually control the weight of this mysterious orb?
> 
> Day 34: One of the crewmembers was wandering the rooms today and claimed that the numbers on the door flashed white as he approached!  He said the door still didn't open, but he noticed that the hourglass had run out and flashed black.  When we went to check on it, it was still running like it always does.  Perhaps he is going mad?  If not, which do we need to appease: the door or the hourglass?  Both?
> 
> Day 55: The fireflies are getting suspicious.  One of them looked at me funny today and then flew off.  I think I saw another one blinking a little faster than usual.  Or was it a little slower?  We are getting better at controlling the weight of the orb, and we think that's what the numbers are all about.  The orb starts at the weight labeled on the pedestal, and goes down as we leave a room marked '-', up as we leave a room marked '+', and up even more as we leave a room marked '*'.  Entering rooms with larger numbers has a greater effect.
> 
> Day 89: Every once in a great while, one of the crewmembers has the same story: that the door flashes white, the hourglass had already run out, it flashes black, and the orb evaporates.  Are we too slow?  We can't seem to find a way to make the orb's weight match what the door wants before the hourglass runs out.  If only we could find a shorter route through the rooms...
> 
> Day 144: We are abandoning the mission.  None of us can work out the solution to the puzzle.  I will leave this journal here to help future adventurers, though I am not sure what help it will give.  Good luck!

## The map of rooms

```
         end ^ (30)
 *   8   -   1
 4   *  11   *
 +   4   -  18
22   -   9   *
^
start
```

## Other notes

- orb changes colours during walking on pads (green, yellow, red)
- if you enter to the room with vault doors and orb will not have correct weight, it will disappear
- orb will shatter if weight drop to zero or below (number are limited to 0 - 32768)
- orb seems to shatter if its weight cross certain value, probably around maxium 32768


In [1]:
def parse_pad(c):
    if c == '*':
        return lambda a, b: a * b
    elif c == '-':
        return lambda a, b: a - b
    elif c == '+':
        return lambda a, b: a + b

    return int(c)

MAX_INT = 32768
START_POINT = (0, 3)
END_POINT = (3, 0)
REQUESTED_VALUE = 30

rooms_map = {(x, y): parse_pad(c) for y, line in enumerate(''' *   8   -   1
 4   *  11   *
 +   4   -  18
22   -   9   *'''.splitlines()) for x, c in enumerate(line.split())}


The problem is finding the shortest (or rather any path, but the jorney suggest be faster than some unknown timer). The graph (map can understand as the graph) contain the negative weights, so simple approach seems to be a bad one. But, I assume, that resultion can be use typed by human, so I limited the path length.

In [2]:
def get_neighbours(x, y):
    yield (x - 1, y, 'w')
    yield (x + 1, y, 'e')
    yield (x, y - 1, 'n')
    yield (x, y + 1, 's')

def only_existing_neighbours(x, y):
    yield from filter(lambda p: (p[0], p[1]) in rooms_map, get_neighbours(x, y))

def get_all_possibilities(x, y, starting_value):
    yield from (
        (
            (neighbour_pad_x, neighbour_pad_y),
            rooms_map[operation_pad_x, operation_pad_y](starting_value, rooms_map[neighbour_pad_x, neighbour_pad_y]),
            operation_direction + neighbour_direction
        )
        for operation_pad_x, operation_pad_y, operation_direction in only_existing_neighbours(x, y)
        for neighbour_pad_x, neighbour_pad_y, neighbour_direction in only_existing_neighbours(operation_pad_x, operation_pad_y))


In [3]:
def find_path_to_value_30(maximum_path_length):
    possibilities = [(START_POINT, rooms_map[START_POINT], '')]
    while possibilities:
        current_position, current_value, current_path = possibilities.pop()

        # Limit for calculations
        if len(current_path) > maximum_path_length:
            continue

        if current_position == END_POINT:
            if current_value == REQUESTED_VALUE:
                # We find the path
                return current_path
            else:
                # Orb disappears in final room
                continue

        for room, value, path in get_all_possibilities(*current_position, current_value):
            if 0 < value < MAX_INT and room != START_POINT:
                possibilities.append((
                    room,
                    value,
                    current_path + path
                ))

In [4]:
%%time

print('Solution:', find_path_to_value_30(12))

Solution: neenwseewnne
CPU times: user 2.28 s, sys: 28.3 ms, total: 2.31 s
Wall time: 2.31 s
